<a href="https://colab.research.google.com/github/Tttthea/speaker/blob/main/sd_classfifier_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#@title install packages
!pip install -qq torch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0 torchtext==0.12.0
!pip install -qq speechbrain==0.5.12
!pip install -qq pyannote.audio
!pip install pydub
!pip install librosa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 102.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 1.11.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.8/496.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 43.3 MB/s eta 0:00:00
     ━━━━━━━

In [3]:
#@title import dependencies
import pandas as pd
import numpy as np
import json
import os
import scipy.io.wavfile
import matplotlib.pyplot as plt
from IPython.display import Audio
import IPython.display as ipd
import scipy.misc
import librosa
import librosa.display
from pydub import AudioSegment
from pyannote.audio import Pipeline
import shutil

In [4]:
#@title speaker diarization data(ICSI)
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!wget -P Signals/Bdb001 https://groups.inf.ed.ac.uk/ami//ICSIsignals/NXT/Bdb001.interaction.wav
sd_path = "../Signals/Bdb001"
audio_wav = f'{sd_path}/Bdb001.interaction.wav'

--2023-04-28 10:50:15--  https://groups.inf.ed.ac.uk/ami//ICSIsignals/NXT/Bdb001.interaction.wav
Resolving groups.inf.ed.ac.uk (groups.inf.ed.ac.uk)... 129.215.202.26
Connecting to groups.inf.ed.ac.uk (groups.inf.ed.ac.uk)|129.215.202.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96701996 (92M) [audio/x-wav]
Saving to: ‘Signals/Bdb001/Bdb001.interaction.wav’

Bdb001.interaction. 100%[===================>]  92.22M  5.38MB/s    in 17s     

2023-04-28 10:50:33 (5.59 MB/s) - ‘Signals/Bdb001/Bdb001.interaction.wav’ saved [96701996/96701996]



In [5]:
#@title connect github repository
!git clone https://github.com/Tttthea/speaker.git
%cd speaker/

Cloning into 'speaker'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 31 (delta 10), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (31/31), 1.71 MiB | 5.60 MiB/s, done.
/content/speaker


In [6]:
import os
import pickle
import SpkrDiarization
from helper import to_wav, out
from preprocessing import feat_engineering

In [ ]:
#@title audio path
path = audio_wav
with open("model/model.pickle", 'rb') as f:
    model = pickle.load(f)
if path.split(".")[-1] != "wav":
    path = to_wav(path)
sp = SpkrDiarization.exportAudio(path)
df = pd.DataFrame(columns=[i for i in range(72)])
outs = pd.DataFrame(columns=['start', 'end', 'speaker', 'predicted', 'clip_path'])
if os.path.exists(sp):
    for audio in os.listdir(sp):
        str = audio.split("_")
        start = int(str[2]) / 1000
        end = int(str[3]) / 1000
        speaker = audio.split("_")[0] + "_" + audio.split("_")[1]
        clip_path = path.split(".")[0] + "/" + audio
        line = feat_engineering(path, False)
        df = df.append(line, ignore_index=True)
        outs.loc[len(outs.index)] = [start, end, speaker, None, clip_path]
outs['predicted'] = model.predict(df)
outs['predicted'] = outs['predicted'].apply(lambda x: "male" if x == 1 else "female")
print(outs)
outs.to_csv('./outs.csv')

[10:51:11] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[10:51:11] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.



using 193.7s generating pipeline
using 0.001372s iterating
using 0.0001721s put into speaker list
{'SPEAKER_04': [[0.5, 6.34], [8.24, 9.68], [16.38, 18.96], [20.24, 23.5], [32.68, 35.18], [46.57, 47.28], [56.51, 59.21], [112.06, 125.68], [127.84, 128.12], [131.6, 132.24], [132.87, 141.27], [141.94, 155.12], [157.65, 165.0], [165.99, 187.24], [189.82, 217.07], [218.19, 222.57], [226.15, 229.31], [231.09, 232.09], [233.59, 236.55], [240.29, 247.08], [248.59, 250.05], [253.54, 255.66], [257.74, 259.33], [260.81, 273.27], [276.52, 288.39], [347.82, 351.06], [353.19, 355.02], [357.78, 357.94], [361.2, 368.29], [375.51, 377.1], [378.35, 379.58], [387.78, 393.72], [406.85, 424.85], [434.49, 448.7], [451.11, 465.44], [469.02, 479.14], [489.96, 502.36], [532.41, 533.66], [535.76, 547.45], [557.14, 604.18], [606.56, 612.4], [620.79, 625.5], [627.74, 634.34], [652.78, 657.15], [663.96, 666.91], [668.31, 668.44], [684.54, 685.05], [692.66, 693.03], [695.19, 704.17], [705.89, 706.38], [777.07, 815.

In [ ]:
#ram crushed -> 314 audio clips, 314 predictions